In [1]:
!pip install -qU pydantic_ai
!pip install -qU langchain langchain-community langchain-huggingface faiss-cpu
!pip install -qU sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.37.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
import os
import getpass
import nest_asyncio #https://ai.pydantic.dev/troubleshooting/
nest_asyncio.apply()

os.environ["OPENAI_KEY"] = getpass.getpass("OpenAI API key :")
#os.environ["GOOGLE_KEY"] = getpass.getpass("Google AI Studio API key :")

OpenAI API key :··········


In [3]:
# IMPORTS
from datetime import datetime

from pydantic import BaseModel
from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.models.gemini import GeminiModel

In [9]:
# MODEL DEFINITION
llm_openrouter = 'google/gemini-2.0-flash-exp:free'
llm_google = 'gemini-2.0-flash-exp'

# model = GeminiModel(llm_google, api_key = os.getenv('GOOGLE_KEY'))
model = OpenAIModel(
    llm_openrouter,
    base_url = 'https://openrouter.ai/api/v1',
    api_key=os.getenv('OPENAI_KEY')
)

# SYSTEMP PROMPT
system_prompt = """
You are a coding expert agent for 4GL language used for SAGE X3 ERP.

Your job is to answer the user query, with example of 4GL code if needed.

Check your tools to query the SAGE X3 ERP knowledge base.

If you don't know the answer, say it - don't try to make it up!

Always return an answer to the user query.

"""

# INSTANCIATE AGENT
agent = Agent(
    model,
    system_prompt=system_prompt,
    retries=2
)

In [10]:
# TOOL 1 : DUMMY RAG FOR TESTING
@agent.tool_plain
def query_knowledge_base() -> str:
    """Query the knowledge base to find relevant 4GL syntaxe.

    Returns:
        str: L4G documentation formatted as string.
    """

    print("Using query_knowledge_base() tool...")

    data = """
    In Sage X3, declaring a variable and assigning it cannot be done on the same line directly.
same line directly. This constraint is designed to clarify the code and avoid the programming errors
programming errors common in more permissive environments.
Scope :
Defines whether the variable is local or global. Local variables exist only in the block
in the block where they are defined, whereas global variables remain active for the entire
session.
Type:
Type of variable, such as Integer, Char, Date, etc.
Variable Name: The name assigned to the variable, following Sage X3 naming conventions.
Dimension:
For tables, indicates the number of elements or indexing limits.
Declaration examples :
Local Integer Without Dimension:
“Local Integer counter”
Declares a local integer variable with no specific dimension, typically used for
counters or other simple numerical values.
Char Global With Dimension:
“Global Char codeProduct[10]”
Declares a global variable of character type with a length of 10 characters, useful for storing
codes or fixed identifiers.
Local Date:
“Local Date dateInscription”
Declares a local date variable, used to store dates such as the registration date.
registration.
Local Decimal With Multiple Dimension:
“Local Decimal price[10]”
    """

    return data

In [11]:
# RUN AGENT
result = agent.run_sync('How do you declare a variable in SAGE X3 ERP?')
print(result.data)

Using query_knowledge_base() tool...
In Sage X3, you declare variables using the following syntax:

```4gl
   Local <data_type> <variable_name> [<dimension>]
   Global <data_type> <variable_name> [<dimension>]
```

*   **Local** or **Global**: Specifies the scope of the variable. Local variables are only accessible within the current block of code, while global variables are accessible throughout the entire program.
*   **data\_type**: Specifies the type of data the variable will store (e.g., Integer, Char, Date, Decimal, etc.).
*   **variable\_name**: The name you give to the variable.
*   **dimension**: (Optional) For arrays or tables, this specifies the size or number of elements.

Here are a few examples:

```4gl
Local Integer counter  # Declares a local integer variable
Global Char codeProduct[10] # Declares a global character variable with a length of 10
Local Date dateInscription # Declares a local date variable
Local Decimal price[10] # Declares a local decimal variable as an a